In [1]:
import pandas as pd
import numpy

clean_data = pd.read_csv('clean_data.csv')

clean_data.head()

,Unnamed: 0,IMO,Anchorage,Port,Berth or Port Entry,Waiting Time (Days),Vessel Type,Date,Zone,Country,...,Panamax (DWT),Panamax Laden (DWT),Panamax Unladen (DWT),Handymax (DWT),Handymax Laden (DWT),Handymax Unladen (DWT),Handysize (DWT),Handysize Laden (DWT),Handysize Unladen (DWT),Wait Levels
0,4,9056399,Santos Anchorage,Santos,2015-01-02 16:34:00,0.2,3,2015-01-02,East Coast South America,Sao Paulo,...,"392,746",0,"392,746","432,704","121,785","310,919","52,823","30,060","22,763",0
1,7,8309141,Santos Anchorage,Santos,2015-01-03 22:22:00,0.5,1,2015-01-03,East Coast South America,Sao Paulo,...,"363,888",0,"363,888","474,533","107,174","367,359","70,297","36,036","34,261",0
2,13,9471252,Santos Anchorage,Santos,2015-01-06 07:15:00,3.4,2,2015-01-02,East Coast South America,Sao Paulo,...,"392,746",0,"392,746","432,704","121,785","310,919","52,823","30,060","22,763",1
3,14,9667435,Santos Anchorage,Santos,2015-01-07 03:46:00,1.9,3,2015-01-05,East Coast South America,Sao Paulo,...,"414,800",0,"414,800","661,542","123,673","537,869","88,003","30,060","57,943",0
4,15,9625970,Santos Anchorage,Santos,2015-01-07 05:46:00,3.9,2,2015-01-03,East Coast South America,Sao Paulo,...,"363,888",0,"363,888","474,533","107,174","367,359","70,297","36,036","34,261",1


In [2]:
#Function for Processing Date

import datetime, re, sys, holidays

def process_date(input_str: str) -> {}:
    # Validate date string input
    regex = re.compile(r'\d{4}-\d{2}-\d{2}')
    if not re.match(regex, input_str):
        print("Invalid date format")
        sys.exit(1)

    # Process date features
    my_date = datetime.datetime.strptime(input_str, '%Y-%m-%d').date()
    now = datetime.datetime.now().date()
    date_feats = {}

    date_feats['date'] = input_str
    date_feats['year'] = my_date.strftime('%Y')
    date_feats['year_s'] = my_date.strftime('%y')
    date_feats['month_num'] = my_date.strftime('%m')
    date_feats['month_text_l'] = my_date.strftime('%B')
    date_feats['month_text_s'] = my_date.strftime('%b')
    date_feats['dom'] = my_date.strftime('%d')
    date_feats['doy'] = my_date.strftime('%j')
    date_feats['woy'] = my_date.strftime('%W')

    # Fixing day of week to start on Mon (1), end on Sun (7)
    dow = my_date.strftime('%w')
    if dow == '0': dow = 7
    date_feats['dow_num'] = dow

    if dow == '1':
        date_feats['dow_text_l'] = 'Monday'
        date_feats['dow_text_s'] = 'Mon'
    if dow == '2':
        date_feats['dow_text_l'] = 'Tuesday'
        date_feats['dow_text_s'] = 'Tue'
    if dow == '3':
        date_feats['dow_text_l'] = 'Wednesday'
        date_feats['dow_text_s'] = 'Wed'
    if dow == '4':
        date_feats['dow_text_l'] = 'Thursday'
        date_feats['dow_text_s'] = 'Thu'
    if dow == '5':
        date_feats['dow_text_l'] = 'Friday'
        date_feats['dow_text_s'] = 'Fri'
    if dow == '6':
        date_feats['dow_text_l'] = 'Saturday'
        date_feats['dow_text_s'] = 'Sat'
    if dow == '7':
        date_feats['dow_text_l'] = 'Sunday'
        date_feats['dow_text_s'] = 'Sun'

    if int(dow) > 5:
        date_feats['is_weekday'] = False
        date_feats['is_weekend'] = True
    else:
        date_feats['is_weekday'] = True
        date_feats['is_weekend'] = False

    # Check date in relation to holidays
    br_holidays = holidays.Brazil()
    date_feats['is_holiday'] = input_str in br_holidays
    date_feats['is_day_before_holiday'] = my_date + datetime.timedelta(days=1) in br_holidays
    date_feats['is_day_after_holiday'] = my_date - datetime.timedelta(days=1) in br_holidays


    return date_feats

In [3]:
dates = clean_data['Date'].astype(str)

date_df = pd.DataFrame()

for d in dates:
    my_date = process_date(d)
    features = [my_date['date'],
                my_date['year'],
                my_date['month_num'],
                my_date['month_text_s'],
                my_date['dom'],
                my_date['doy'],
                my_date['woy'],
                my_date['is_weekend'],
                my_date['is_holiday']]
    ds = pd.Series(features)
    date_df = date_df.append(ds, ignore_index=True)

date_df.rename(columns={0: 'Date',
                   1: 'year',
                   2: 'month_num',
                   3: 'month',
                   4: 'day_of_month',
                   5: 'day_of_year',
                   6: 'week_of_year',
                   7: 'is_weekend',
                   8: 'is_holiday'}, inplace=True)

C:\Users\rawwi\AppData\Local\Temp\ipykernel_18336\3657349985.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  date_df = date_df.append(ds, ignore_index=True)
C:\Users\rawwi\AppData\Local\Temp\ipykernel_18336\3657349985.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  date_df = date_df.append(ds, ignore_index=True)


In [4]:
date_df = date_df.drop(['Date'], axis = 1)
date_df.reset_index(drop=True, inplace=True)
clean_data.reset_index(drop=True, inplace=True)

In [5]:
clean_data = pd.concat([clean_data, date_df], axis = 1)

clean_data

,Unnamed: 0,IMO,Anchorage,Port,Berth or Port Entry,Waiting Time (Days),Vessel Type,Date,Zone,Country,...,Handysize Unladen (DWT),Wait Levels,year,month_num,month,day_of_month,day_of_year,week_of_year,is_weekend,is_holiday
0,4,9056399,Santos Anchorage,Santos,2015-01-02 16:34:00,0.2,3,2015-01-02,East Coast South America,Sao Paulo,...,"22,763",0,2015,01,Jan,02,002,00,False,False
1,7,8309141,Santos Anchorage,Santos,2015-01-03 22:22:00,0.5,1,2015-01-03,East Coast South America,Sao Paulo,...,"34,261",0,2015,01,Jan,03,003,00,True,False
2,13,9471252,Santos Anchorage,Santos,2015-01-06 07:15:00,3.4,2,2015-01-02,East Coast South America,Sao Paulo,...,"22,763",1,2015,01,Jan,02,002,00,False,False
3,14,9667435,Santos Anchorage,Santos,2015-01-07 03:46:00,1.9,3,2015-01-05,East Coast South America,Sao Paulo,...,"57,943",0,2015,01,Jan,05,005,01,False,False
4,15,9625970,Santos Anchorage,Santos,2015-01-07 05:46:00,3.9,2,2015-01-03,East Coast South America,Sao Paulo,...,"34,261",1,2015,01,Jan,03,003,00,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,9971,9127071,Santos Anchorage,Santos,2021-12-30 23:28:00,8.3,1,2021-12-22,East Coast South America,Sao Paulo,...,"38,663",2,2021,12,Dec,22,356,51,False,False
9943,9972,9747558,Santos Anchorage,Santos,2021-12-31 01:41:00,65.3,2,2021-10-26,East Coast South America,Sao Paulo,...,"70,144",3,2021,10,Oct,26,299,43,False,False
9944,9973,9867059,Santos Anchorage,Santos,2021-12-31 12:47:00,15.9,2,2021-12-15,East Coast South America,Sao Paulo,...,"33,985",3,2021,12,Dec,15,349,50,False,False
9945,9974,9566447,Santos Anchorage,Santos,2021-12-31 12:53:00,1.7,3,2021-12-29,East Coast South America,Sao Paulo,...,"177,554",0,2021,12,Dec,29,363,52,False,False


In [6]:
weather_df = pd.read_csv('sao_paulo_weather.csv')
weather_df = weather_df[['date','tavg','prcp']]
weather_df = weather_df.fillna(0)

weather_df.columns = ['Date', 'tempt', 'prcp']

weather_df.head()

,Date,tempt,prcp
0,2015-01-01,26.8,0.0
1,2015-01-02,27.0,0.0
2,2015-01-03,23.3,3.0
3,2015-01-04,22.9,0.0
4,2015-01-05,22.7,2.0


In [7]:
clean_data = pd.merge(clean_data, weather_df, on='Date', how='left')

In [8]:
clean_data

,Unnamed: 0,IMO,Anchorage,Port,Berth or Port Entry,Waiting Time (Days),Vessel Type,Date,Zone,Country,...,year,month_num,month,day_of_month,day_of_year,week_of_year,is_weekend,is_holiday,tempt,prcp
0,4,9056399,Santos Anchorage,Santos,2015-01-02 16:34:00,0.2,3,2015-01-02,East Coast South America,Sao Paulo,...,2015,01,Jan,02,002,00,False,False,27.0,0.0
1,7,8309141,Santos Anchorage,Santos,2015-01-03 22:22:00,0.5,1,2015-01-03,East Coast South America,Sao Paulo,...,2015,01,Jan,03,003,00,True,False,23.3,3.0
2,13,9471252,Santos Anchorage,Santos,2015-01-06 07:15:00,3.4,2,2015-01-02,East Coast South America,Sao Paulo,...,2015,01,Jan,02,002,00,False,False,27.0,0.0
3,14,9667435,Santos Anchorage,Santos,2015-01-07 03:46:00,1.9,3,2015-01-05,East Coast South America,Sao Paulo,...,2015,01,Jan,05,005,01,False,False,22.7,2.0
4,15,9625970,Santos Anchorage,Santos,2015-01-07 05:46:00,3.9,2,2015-01-03,East Coast South America,Sao Paulo,...,2015,01,Jan,03,003,00,True,False,23.3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,9971,9127071,Santos Anchorage,Santos,2021-12-30 23:28:00,8.3,1,2021-12-22,East Coast South America,Sao Paulo,...,2021,12,Dec,22,356,51,False,False,23.9,0.0
9943,9972,9747558,Santos Anchorage,Santos,2021-12-31 01:41:00,65.3,2,2021-10-26,East Coast South America,Sao Paulo,...,2021,10,Oct,26,299,43,False,False,18.4,0.0
9944,9973,9867059,Santos Anchorage,Santos,2021-12-31 12:47:00,15.9,2,2021-12-15,East Coast South America,Sao Paulo,...,2021,12,Dec,15,349,50,False,False,22.4,0.5
9945,9974,9566447,Santos Anchorage,Santos,2021-12-31 12:53:00,1.7,3,2021-12-29,East Coast South America,Sao Paulo,...,2021,12,Dec,29,363,52,False,False,21.2,0.0


In [10]:
clean_data.dtypes

Unnamed: 0                   int64
IMO                          int64
Anchorage                   object
Port                        object
Berth or Port Entry         object
Waiting Time (Days)        float64
Vessel Type                  int64
Date                        object
Zone                        object
Country                     object
Total Vessels (Number)     float64
Total Vessels (DWT)         object
Capesize                   float64
Capesize Laden             float64
Capesize Unladen           float64
Panamax                    float64
Panamax Laden              float64
Panamax Unladen            float64
Handymax                     int64
Handymax Laden             float64
Handymax Unladen           float64
Handysize                  float64
Handysize Laden            float64
Handysize Unladen          float64
Capesize (DWT)              object
Capesize Laden (DWT)        object
Capesize Unladen (DWT)      object
Panamax (DWT)               object
Panamax Laden (DWT) 